In [1]:
import importlib
import ldsc_71 as ld
from helperfuncs import extract_upper_triangle, return_to_symmetric, extract_bounds
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
importlib.reload(ld)

<module 'ldsc_71' from 'C:\\Users\\Hariharan\\Documents\\git_repos\\SNIPar\\ldsc_reg\\ldsc_71.py'>

In [2]:
N = 100
S = np.array([np.array([[5, 0], [0, 5]]),
    np.array([[2, 0], [0, 2]])] * 50 )# 50 = N/2
V = np.identity(2) * 10.0


model = ld.sibreg_71(S = S)
model.simdata(V, N)

output_matrix, result = model.solve()

Warning there is no value for theta. Maybe consider simulating it
No initial guess provided.
Making a matrix of 0s as the initial estimate
Final Estimate:
 [[8.91734087 0.23379371]
 [0.23379371 8.03524308]]
Convergence Flag:  b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
Number of Iterations:  12
Final Gradient:  [-7.84196579e-08 -1.22227527e-07  2.47605877e-07]
Max deviation of gradient from 0:  2.4760587718011817e-07


In [3]:
def jackknife():
    
    # Simple jackknife estimator for SE
    # Source: https://www.stat.berkeley.edu/~hhuang/STAT152/Jackknife-Bootstrap.pdf
    
    theta = model.theta
    S = model.S
    
    nobs = theta.shape[0]
    
    estimates_jk = []
    
    for i in range(nobs):
        
        # remove ith observation
        theta_jk = theta[np.arange(len(theta)) != i]
        S_jk = S[np.arange(len(S)) != i]
        
        # Get our estimate
        output_matrix, _ = model.solve(theta = theta_jk,
                                      S = S_jk,
                                      printout = False)
        
        estimates_jk.append(output_matrix)
        
    estimates_jk = np.array(estimates_jk)
    
    # calculate the SE
    estimate_jk_mean = estimates_jk.mean(axis = 0)
    estimate_jk_mean = np.array([estimate_jk_mean] * nobs) 
    estimates_jk_dev = estimates_jk - estimate_jk_mean
    estimates_jk_devsq = estimates_jk_dev ** 2
    estimates_jk_devsq_sum = estimates_jk_devsq.sum(axis = 0)

    se = ((nobs - 1)/nobs) * estimates_jk_devsq_sum
    se = np.sqrt(se)
    
    return se  

In [4]:
jackknife()

array([[1.66970423, 1.33436631],
       [1.33436631, 1.4582945 ]])